<a href="https://colab.research.google.com/github/crisalex25/TFM_HydroMLGenerationOptimization/blob/main/TFM_Optimizaci%C3%B3n_de_la_generaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabajo de Fin de Master**
## **Máster Universitario en Inteligencia Artificial para el Sector de la Energía y las Infraestructuras**
Optimización de generación en centrales hidroeléctricas mediante técnicas de aprendizaje automático

##Optimización

Librerías

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.3 MB/s eta 0:00:00


In [ ]:
'''
    LIBRERIAS
    ---------------------------------------------------------
'''
# Librerías de Algoritmos genéticos
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

# Librerías auxiliares
import matplotlib.pyplot as plt

# Librerías auxiliares
from operator import truediv
import numpy as np
import random
import time

In [ ]:
#Registro el tiempo de inicio del programa
t0 = time.time()

Variables y parámetros

In [ ]:
numeroGenesTotal= 72
numeroCromosomas=3
numeroGenesPorCromosoma= int(numeroGenesTotal/numeroCromosomas)
probCruce=0.7            #Probabilidad de cruce
probMutacion=0.3         #Probabilidad de mutación
num_generaciones= 500    #Numero de generaciones
tamaño_poblacion = 500   #Tamaño de la población
penaliza=9999999.9

P_min_G1=400.0    #Potencia mínima que puede producir el generador 1 Papallacta
P_max_G1=1800.0   #Potencia máxima que puede producir el generador 1 Papallacta
P_max_G2=4200.0   #Potencia máxima que puede producir el generador 2 Papallacta
P_min_G2=1700.0   #Potencia mínima que puede producir el generador 2 Papallacta
P_max_G3=2100.0   #Potencia máxima que puede producir el generador 3 Loreto
P_min_G3=500.0   #Potencia mínima que puede producir el generador 3 Loreto

#Demanda de potencia energética que se debe satisfacer
demandaConsumoPorHoras_GC = [1200, 1333, 1444, 1884, 1884, 1884, 1884, 1884, 1884, 1484, 1584, 1284, 1684, 1784, 1684, 1114, 1284, 1384, 1884, 1884, 1884, 1884, 1884, 1700]
demandaConsumoPorHoras_AC = [4200, 4333, 4444, 4884, 4884, 4884, 4884, 5884, 5884, 5484, 5584, 5284, 5684, 5784, 5684, 6114, 6284, 6384, 5884, 5484, 5884, 5884, 5684, 5300]
horas = [0,1,2,3,4, 5, 6, 7, 8, 9, 10,11,12,13,14,15, 16,17,18,19,20,21,22,23]

Funciones Auxiliares

In [ ]:
#Sección que contiene funciones auxiliares propias del dominio del problema a resolver

#Función que obtiene el coste en € que correspondiente al generador diesel de producir una potencia determinada
def getCosteDiesel(potencia):
    return float(((potencia * potencia) / 50) * 1.8)

#Función que obtiene el coste en € que correspondiente al generador de gas de producir una potencia determinada
def getCosteGas(potencia):
    return float(potencia * 1.04)

#Función que muestra un gráfico con los datos de las potencias contenidas en un individuo.
def dibujarGrafico(individuo, demandaConsumoPorHoras, horas,tituloGrafico):
        individuo1 = np.zeros(numeroGenesPorCromosoma)
        for i in range(0,numeroGenesPorCromosoma):
            individuo1[i]=individuo[i] #Generador G1 Papallacta

        individuo2 = np.zeros(numeroGenesPorCromosoma)
        for i in range(0,numeroGenesPorCromosoma):
            individuo2[i]=individuo[i+numeroGenesPorCromosoma] #Generador G2 Papallacta

        individuo3 = np.zeros(numeroGenesPorCromosoma)
        for i in range(0,numeroGenesPorCromosoma):
            individuo3[i]=individuo[i+numeroGenesPorCromosoma*2] #Generador G3 Loreto

        plt.bar(horas, individuo1 + individuo2 , label='G2',color='green')
        plt.bar(horas, individuo1 , label='G1',color='blue')
        plt.plot(horas,demandaConsumoPorHoras_AC,'*--', label='Total' )
        plt.ylabel('Potencia (Kw)')
        plt.xlabel('Horas (0-24)')
        plt.title(tituloGrafico)
        #agregar segundo grafico

        plt.legend(loc='best')
        plt.show()

#La función realiza varias comprobaciones para saber si un individuo es válido.
def individuoIsValid(x):

    #Elimino valores negativos
    for i in range(0,numeroGenesPorCromosoma*3):
        if (x[i] < float(0.0)):
            return False

    #El generador G1 debe generar más potencia de 400 kw para que funcione
    for i in range(0,numeroGenesPorCromosoma):
        if (x[i]>0.0 and x[i]< P_min_G1):
            return False

    #El generador G2 debe generar más potencia de 1700 kw para que funcione
    for i in range(numeroGenesPorCromosoma,numeroGenesPorCromosoma*2):
        if (x[i]>0.0 and x[i]< P_min_G2):
            return False

    #Compruebo que el generador de Diesel solo puede tener asignada como máxima su maximo permitido
    for i in range(0,numeroGenesPorCromosoma):
        if (x[i] > P_max_GD):
            return False

    #Compruebo que el generador de gas solo puede tener asignada como máxima su maximo permitido
    for i in range(numeroGenesPorCromosoma,numeroGenesPorCromosoma*2):
       if (x[i] > P_max_GG):
            return False

    #Compruebo si el individuo satisface la demanda energética
    for i in range(len(demandaConsumoPorHoras)):
        demandaAsatistacer = float(demandaConsumoPorHoras[i])  - float(energiaRenovableSolar[i])
        diferencia= demandaAsatistacer - float(x[i]) - float(x[i+numeroGenesPorCromosoma])
        if (diferencia > 0.0):
            return False
    return True

###Optimización Ejemplos

In [ ]:
import random
from tabulate import tabulate

def calcular_costo_deficit(generacion, demanda, costo_horario):
    deficit = generacion - demanda
    if deficit < 0:
        costo_deficit = abs(deficit) * costo_horario
    else:
        costo_deficit = 0
    return costo_deficit

def generar_solucion_inicial(capacidad_unit1, capacidad_unit2, rango_horario):
    generacion_inicial = {}
    for hora in rango_horario:
        generacion_unit1 = random.uniform(400, capacidad_unit1)
        generacion_unit2 = random.uniform(1600, capacidad_unit2)
        generacion_inicial[hora] = (generacion_unit1, generacion_unit2)
    return generacion_inicial

def generar_movimiento(generacion_unit):
    incremento = random.uniform(-0.1, 0.1) * generacion_unit
    nueva_generacion_unit = generacion_unit + incremento
    return max(nueva_generacion_unit, 0)  # Asegurarse de no tener generación negativa

def aplicar_heuristica(capacidad_unit1, capacidad_unit2, demanda_energia, costo_horario, rango_horario, iteraciones):
    mejor_costo_deficit = float('inf')
    mejor_generacion = {}

    for _ in range(iteraciones):
        generacion = generar_solucion_inicial(capacidad_unit1, capacidad_unit2, rango_horario)
        costo_deficit = sum(calcular_costo_deficit(generacion[hora][0] + generacion[hora][1], demanda_energia[hora], costo_horario) for hora in rango_horario)

        for _ in range(100):  # Número fijo de iteraciones de mejora por solución inicial
            nueva_generacion = {}
            for hora in rango_horario:
                generacion_unit1, generacion_unit2 = generacion[hora]
                nueva_generacion_unit1 = generar_movimiento(generacion_unit1)
                nueva_generacion_unit2 = generar_movimiento(generacion_unit2)
                nueva_generacion[hora] = (nueva_generacion_unit1, nueva_generacion_unit2)

            nuevo_costo_deficit = sum(calcular_costo_deficit(nueva_generacion[hora][0] + nueva_generacion[hora][1], demanda_energia[hora], costo_horario) for hora in rango_horario)

            if nuevo_costo_deficit < costo_deficit:
                generacion = nueva_generacion
                costo_deficit = nuevo_costo_deficit
            else:
                break

        if costo_deficit < mejor_costo_deficit:
            mejor_costo_deficit = costo_deficit
            mejor_generacion = generacion

    return mejor_generacion, mejor_costo_deficit

# Ejemplo de uso
capacidad_unit1 = 1800
capacidad_unit2 = 4200
costo_horario = 10
rango_horario = range(24)  # Horas del día (0 a 23)

# Generar demanda de energía aleatoria para cada hora del día
demanda_energia = {hora: random.uniform(2000, 3600) for hora in rango_horario}

iteraciones = 100

mejor_generacion, mejor_costo_deficit = aplicar_heuristica(capacidad_unit1, capacidad_unit2, demanda_energia, costo_horario, rango_horario, iteraciones)

# Preparar los datos para la tabla
tabla_resultados = []
for hora, generacion in mejor_generacion.items():
    fila = [hora, generacion[0], generacion[1], calcular_costo_deficit(generacion[0] + generacion[1], demanda_energia[hora], costo_horario), demanda_energia[hora]]
    tabla_resultados.append(fila)

# Mostrar los resultados en una tabla
headers = ["Hora", "Generación Unidad 1", "Generación Unidad 2", "Costo Déficit", "Demanda"]
tabla = tabulate(tabla_resultados, headers, tablefmt="grid")
print(tabla)



+--------+-----------------------+-----------------------+-----------------+-----------+
|   Hora |   Generación Unidad 1 |   Generación Unidad 2 |   Costo Déficit |   Demanda |
+========+=======================+=======================+=================+===========+
|      0 |               418.487 |               3493.55 |               0 |   2476.18 |
+--------+-----------------------+-----------------------+-----------------+-----------+
|      1 |              1515.73  |               1747.9  |               0 |   3141.62 |
+--------+-----------------------+-----------------------+-----------------+-----------+
|      2 |               848.682 |               1775.07 |               0 |   2200.58 |
+--------+-----------------------+-----------------------+-----------------+-----------+
|      3 |              1560.96  |               3818.71 |               0 |   3119.75 |
+--------+-----------------------+-----------------------+-----------------+-----------+
|      4 |           

In [ ]:
import random
from tabulate import tabulate

def calcular_costo_deficit(generacion, demanda, costo_horario):
    deficit = generacion - demanda
    if deficit < 0:
        costo_deficit = abs(deficit) * costo_horario
    else:
        costo_deficit = 0
    return costo_deficit

def generar_solucion_inicial(capacidad_unit1, capacidad_unit2, capacidad_unit3, rango_horario):
    generacion_inicial = {}
    for hora in rango_horario:
        generacion_unit1 = random.uniform(0, capacidad_unit1)
        generacion_unit2 = random.uniform(0, capacidad_unit2)
        generacion_unit3 = random.uniform(0, capacidad_unit3)
        generacion_inicial[hora] = (generacion_unit1, generacion_unit2, generacion_unit3)
    return generacion_inicial

def generar_movimiento(generacion_unit):
    incremento = random.uniform(-0.1, 0.1) * generacion_unit
    nueva_generacion_unit = generacion_unit + incremento
    return max(nueva_generacion_unit, 0)  # Asegurarse de no tener generación negativa

def aplicar_heuristica(capacidad_unit1, capacidad_unit2, capacidad_unit3, demanda_energia, costo_horario, rango_horario, iteraciones):
    mejor_costo_deficit = float('inf')
    mejor_generacion = {}

    for _ in range(iteraciones):
        generacion = generar_solucion_inicial(capacidad_unit1, capacidad_unit2, capacidad_unit3, rango_horario)
        costo_deficit = sum(calcular_costo_deficit(sum(generacion[hora]), demanda_energia[hora], costo_horario) for hora in rango_horario)

        for _ in range(100):  # Número fijo de iteraciones de mejora por solución inicial
            nueva_generacion = {}
            for hora in rango_horario:
                generacion_unit1, generacion_unit2, generacion_unit3 = generacion[hora]
                nueva_generacion_unit1 = generar_movimiento(generacion_unit1)
                nueva_generacion_unit2 = generar_movimiento(generacion_unit2)
                nueva_generacion_unit3 = generar_movimiento(generacion_unit3)
                nueva_generacion[hora] = (nueva_generacion_unit1, nueva_generacion_unit2, nueva_generacion_unit3)

            nuevo_costo_deficit = sum(calcular_costo_deficit(sum(nueva_generacion[hora]), demanda_energia[hora], costo_horario) for hora in rango_horario)

            if nuevo_costo_deficit < costo_deficit:
                generacion = nueva_generacion
                costo_deficit = nuevo_costo_deficit
            else:
                break

        if costo_deficit < mejor_costo_deficit:
            mejor_costo_deficit = costo_deficit
            mejor_generacion = generacion

    return mejor_generacion, mejor_costo_deficit

# Ejemplo de uso
capacidad_unit1 = 1800
capacidad_unit2 = 4200
capacidad_unit3 = 2100  # Capacidad del nuevo generador
costo_horario = 10
rango_horario = range(24)  # Horas del día (0 a 23)

# Generar demanda de energía aleatoria para cada hora del día
# Generar demanda de energía aleatoria para cada hora del día
demanda_energia = {hora: random.uniform(2000, 3600) for hora in rango_horario}

# Generar demanda de energía para la nueva industria
demanda_industria = {hora: random.uniform(500, 2200) for hora in rango_horario}

# Combinar las demandas de las dos industrias
demanda_total = {hora: demanda_energia[hora] + demanda_industria[hora] for hora in rango_horario}

iteraciones = 100

mejor_generacion, mejor_costo_deficit = aplicar_heuristica(capacidad_unit1, capacidad_unit2, capacidad_unit3, demanda_total, costo_horario, rango_horario, iteraciones)

# Preparar los datos para la tabla
tabla_resultados = []
for hora, generacion in mejor_generacion.items():
    generacion_total = sum(generacion)
    demanda_total_hora = demanda_total[hora]
    deficit_industria = max(generacion_total - demanda_industria[hora], 0)
    deficit_energia = max(generacion_total - demanda_energia[hora], 0)
    costo_deficit = calcular_costo_deficit(generacion_total, demanda_total_hora, costo_horario)
    fila = [hora, generacion[0], generacion[1], generacion[2], costo_deficit, deficit_industria, deficit_energia, demanda_total_hora]
    tabla_resultados.append(fila)

# Mostrar los resultados en una tabla
headers = ["Hora", "Generación Unidad 1", "Generación Unidad 2", "Generación Unidad 3", "Costo Déficit", "Déficit Industria", "Déficit Energía", "Demanda Total"]
tabla = tabulate(tabla_resultados, headers, tablefmt="grid")
print(tabla)


+--------+-----------------------+-----------------------+-----------------------+-----------------+---------------------+-------------------+-----------------+
|   Hora |   Generación Unidad 1 |   Generación Unidad 2 |   Generación Unidad 3 |   Costo Déficit |   Déficit Industria |   Déficit Energía |   Demanda Total |
+========+=======================+=======================+=======================+=================+=====================+===================+=================+
|      0 |               608.282 |              3224.59  |              483.767  |         253.085 |            3328.4   |           962.93  |         4341.94 |
+--------+-----------------------+-----------------------+-----------------------+-----------------+---------------------+-------------------+-----------------+
|      1 |              1586.26  |              2651.66  |             1952.73   |           0     |            4751.17  |          2968.53  |         4661.59 |
+--------+-----------------------+

###Optimización PULP

In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 74.7 MB/s eta 0:00:00


In [ ]:
"""
 The Furniture problem from EngSci391 for the PuLP Modeller
 Author: Dr Stuart Mitchell    2007
"""
from pulp import *

Chairs = ["A", "B"]
costs = {"A": 100, "B": 150}
Resources = ["Lathe", "Polisher"]
capacity = {"Lathe": 40, "Polisher": 48}
activity = [  # Chairs
    # A  B
    [1, 2],  # Lathe
    [3, 1.5],  # Polisher
]
activity = makeDict([Resources, Chairs], activity)
prob = LpProblem("Furniture Manufacturing Problem", LpMaximize)
vars = LpVariable.dicts("Number of Chairs", Chairs, lowBound=0)
# objective
prob += lpSum([costs[c] * vars[c] for c in Chairs])
for r in Resources:
    prob += (
        lpSum([activity[r][c] * vars[c] for c in Chairs]) <= capacity[r],
        f"capacity_of_{r}",
    )
prob.writeLP("furniture.lp")
prob.solve()
# Each of the variables is printed with it's value
for v in prob.variables():
    print(v.name, "=", v.varValue)
# The optimised objective function value is printed to the screen
print("Total Revenue from Production = ", value(prob.objective))

Number_of_Chairs_A = 8.0
Number_of_Chairs_B = 16.0
Total Revenue from Production =  3200.0


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
import numpy as np
from scipy.optimize import linprog

# Condiciones de operación
gen_max = [1900, 4200, 2100]
gen_min = [400, 1600, 400]
caudal_x = 0.8
demand_autoconsumo = [4811, 4787, 4778, 4780, 4856, 4878, 4853, 4901, 5082, 5250, 5292, 5305, 5103, 5219, 5266, 5258, 5311, 5138, 4989, 4876, 4745, 4876, 4833, 4845]
demand_gran_consumidor = [2114, 2175, 2095, 2248, 2213, 2170, 2172, 2286, 2289, 2213, 2181, 2303, 2374, 2332, 2200, 2264, 2251, 2162, 2261, 2120, 2028, 2136, 2183, 2198]

# Precios de energía horaria
precios_energia = [6.19, 8.39, 6.19, 6.19, 5.35, 6.48, 5.35, 5.35, 6.19, 6.19, 6.19, 6.48, 6.19, 6.48, 6.19, 6.48, 6.19, 6.19, 9.46, 9.46, 8.96, 6.19, 6.19, 6.19]

# Funciones para calcular el caudal turbimado
def calcular_caudal_turbimado(p):
    qt1 = 0.0009 * p[0] + 0.2624
    qt2 = 0.0008 * p[1] + 0.053
    qt3 = 0.0005 * p[2] - 0.0154
    return [qt1, qt2, qt3]

# Matriz de restricciones para el método de programación lineal
A = np.array([
    [-1, 0, 0],
    [0, -1, 0],
    [0, 0, -1],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])
b = np.array([-gen_min[0], -gen_min[1], -gen_min[2], gen_max[0], gen_max[1], gen_max[2]])

# Optimización por programación lineal
min_cost = float('inf')
opt_gen = None

for t in range(len(demand_autoconsumo)):
    demand_total = demand_autoconsumo[t] + demand_gran_consumidor[t]
    demand_unidad3 = demand_gran_consumidor[t]
    demand_unidad1_2 = demand_autoconsumo[t]

    c = np.array([precios_energia[t], precios_energia[t], precios_energia[t]])  # Costos de compra de energía

    # Restricciones adicionales para el caudal turbimado
    A_extra = np.array([
        [1, 1, -1]
    ])
    b_extra = np.array([caudal_x + demand_unidad3])

    # Combinar las restricciones y restricciones adicionales
    A_combined = np.concatenate((A, A_extra), axis=0)
    b_combined = np.concatenate((b, b_extra), axis=0)

    # Resolver el problema de programación lineal
    res = linprog(c, A_ub=A_combined, b_ub=b_combined, bounds=[(gen_min[0], gen_max[0]), (gen_min[1], gen_max[1]), (gen_min[2], gen_max[2])])

    # Calcular el costo de compra de energía de déficit
    deficit = max(0, demand_total - np.sum(res.x))
    costo_deficit = deficit * precios_energia[t]

    # Actualizar la solución óptima si se encuentra un costo menor
    if costo_deficit < min_cost:
        min_cost = costo_deficit
        opt_gen = res.x

# Imprimir resultados
print("Generación óptima de las unidades hidroeléctricas:")
print("Unidad 1: {:.2f} kW".format(opt_gen[0]))
print("Unidad 2: {:.2f} kW".format(opt_gen[1]))
print("Unidad 3: {:.2f} kW".format(opt_gen[2]))
print("Costo mínimo de compra de energía de déficit: {:.2f}".format(min_cost))



Generación óptima de las unidades hidroeléctricas:
Unidad 1: 400.00 kW
Unidad 2: 1600.00 kW
Unidad 3: 400.00 kW
Costo mínimo de compra de energía de déficit: 24743.75


In [ ]:
from pulp import *

# Datos de las unidades hidroeléctricas
gen_max = [1900, 4200, 2100]  # Generación máxima de cada unidad
gen_min = [0, 1600, 400]  # Generación mínima de cada unidad

# Caudal turbina para cada unidad
def caudal_turbina(p):
    return [0.0009 * p[0] + 0.2624, 0.0008 * p[1] + 0.053, 0.0005 * p[2] - 0.0154]

# Precios de energía horaria
precios_energia = [
    6.19, 8.39, 6.19, 6.19, 5.35, 6.48, 5.35, 5.35, 6.19, 6.19, 6.19, 6.48,
    6.19, 6.48, 6.19, 6.48, 6.19, 6.19, 9.46, 9.46, 8.96, 6.19, 6.19, 6.19
]

# Demanda horaria de autoconsumo y gran consumidor
demanda_autoconsumo = [
    4811, 4787, 4778, 4780, 4856, 4878, 4853, 4901, 5082, 5250, 5292, 5305,
    5103, 5219, 5266, 5258, 5311, 5138, 4989, 4876, 4745, 4876, 4833, 4845
]
demanda_gran_consumidor = [
    2114, 2175, 2095, 2248, 2213, 2170, 2172, 2286, 2289, 2213, 2181, 2303,
    2374, 2332, 2200, 2264, 2251, 2162, 2261, 2120, 2028, 2136, 2183, 2198
]

# Crear el problema de optimización
prob = LpProblem("Optimización de generación", LpMinimize)

# Variables de decisión: generación de cada unidad en cada hora
generacion = [
    [LpVariable(f"gen_{i}_{j}", gen_min[i], gen_max[i]) for j in range(24)]
    for i in range(3)
]
# Variable de déficit para el Autoconsumo
deficit_autoconsumo = [LpVariable(f"deficit1_{j}", lowBound=0) for j in range(24)]

# Variable de déficit para el gran consumidor
deficit_gran_consumidor = [LpVariable(f"deficit2_{j}", lowBound=0) for j in range(24)]


# Función objetivo: minimizar el costo de compra de energía de déficit
prob += lpSum([
    precios_energia[j] * deficit_gran_consumidor[j] + precios_energia[j] * deficit_autoconsumo[j]
    for j in range(24)
])

# Restricciones de generación y demanda
for j in range(24):
    # Restricciones de generación mínima y máxima
    for i in range(3):
        prob += generacion[i][j] >= gen_min[i]
        prob += generacion[i][j] <= gen_max[i]

    # Restricción de caudal para unidades 1 y 2
    prob += generacion[0][j] + generacion[1][j] <= generacion[2][j] + 0.8

    # Restricción de demanda de autoconsumo
    #prob += generacion[0][j] + generacion[1][j] >= demanda_autoconsumo[j]

    # Restricción de demanda del gran consumidor
    #prob += generacion[2][j] >= demanda_gran_consumidor[j] + deficit_gran_consumidor[j]

# Resolver el problema de optimización
prob.solve()

# Imprimir el resultado
print("Estado de la solución:", LpStatus[prob.status])
print("Costo de compra de energía de déficit:", value(prob.objective))

# Imprimir la generación óptima de cada unidad en cada hora
for i in range(3):
    print(f"Generación óptima de U{i+1}:")
    for j in range(24):
        print(f"Hora {j+1}: {value(generacion[i][j])}")



Estado de la solución: Optimal
Costo de compra de energía de déficit: 0.0
Generación óptima de U1:
Hora 1: 0.0
Hora 2: 0.0
Hora 3: 0.0
Hora 4: 0.0
Hora 5: 0.0
Hora 6: 0.0
Hora 7: 0.0
Hora 8: 0.0
Hora 9: 0.0
Hora 10: 0.0
Hora 11: 0.0
Hora 12: 0.0
Hora 13: 0.0
Hora 14: 0.0
Hora 15: 0.0
Hora 16: 0.0
Hora 17: 0.0
Hora 18: 0.0
Hora 19: 0.0
Hora 20: 0.0
Hora 21: 0.0
Hora 22: 0.0
Hora 23: 0.0
Hora 24: 0.0
Generación óptima de U2:
Hora 1: 1600.0
Hora 2: 1600.0
Hora 3: 1600.0
Hora 4: 1600.0
Hora 5: 1600.0
Hora 6: 1600.0
Hora 7: 1600.0
Hora 8: 1600.0
Hora 9: 1600.0
Hora 10: 1600.0
Hora 11: 1600.0
Hora 12: 1600.0
Hora 13: 1600.0
Hora 14: 1600.0
Hora 15: 1600.0
Hora 16: 1600.0
Hora 17: 1600.0
Hora 18: 1600.0
Hora 19: 1600.0
Hora 20: 1600.0
Hora 21: 1600.0
Hora 22: 1600.0
Hora 23: 1600.0
Hora 24: 1600.0
Generación óptima de U3:
Hora 1: 2100.0
Hora 2: 2100.0
Hora 3: 2100.0
Hora 4: 2100.0
Hora 5: 2100.0
Hora 6: 2100.0
Hora 7: 2100.0
Hora 8: 2100.0
Hora 9: 2100.0
Hora 10: 2100.0
Hora 11: 2100.0
Hora 1

In [ ]:

from pulp import *
from pulp import *

# Datos de entrada
max_generacion = [1900, 4200, 2100]  # Generación máxima de cada unidad
min_generacion = [400, 1600, 400]  # Generación mínima de cada unidad
demanda_autoconsumo = [4811, 4787, 4778, 4780, 4856, 4878, 4853, 4901, 5082, 5250, 5292, 5305, 5103, 5219, 5266, 5258, 5311, 5138, 4989, 4876, 4745, 4876, 4833, 4845]
demanda_gran_consumidor = [2114, 2175, 2095, 2248, 2213, 2170, 2172, 2286, 2289, 2213, 2181, 2303, 2374, 2332, 2200, 2264, 2251, 2162, 2261, 2120, 2028, 2136, 2183, 2198]

precio_energia = [6.19, 8.39, 6.19, 6.19, 5.35, 6.48, 5.35, 5.35, 6.19, 6.19, 6.19, 6.48, 6.19, 6.48, 6.19, 6.48, 6.19, 6.19, 9.46, 9.46, 8.96, 6.19, 6.19, 6.19]

Qx = 0.8  # Caudal constante

# Verificar la longitud de precio_energia
assert len(precio_energia) == len(demanda_autoconsumo) + len(demanda_gran_consumidor), "Longitud de precio_energia incorrecta"

# Crear problema de optimización
prob = LpProblem("OptimizacionGeneracion", LpMinimize)

# Variables de decisión
generacion = []
for i in range(3):
    generacion.append(LpVariable(f"GeneracionU{i+1}", lowBound=min_generacion[i], upBound=max_generacion[i], cat='Continuous'))

# Función objetivo
costo_total = lpSum([generacion[i] * precio_energia[j] for i, j in enumerate(range(len(demanda_autoconsumo) + len(demanda_gran_consumidor)))])
prob += costo_total

# Restricciones de generación
for i in range(3):
    prob += generacion[i] >= min_generacion[i]
    prob += generacion[i] <= max_generacion[i]

# Restricciones de autoconsumo y gran consumidor
for t in range(len(demanda_autoconsumo)):
    # Restricciones de autoconsumo
    prob += generacion[0] + generacion[1] >= demanda_autoconsumo[t]

    # Restricciones de gran consumidor
    prob += generacion[2] >= demanda_gran_consumidor[t]

# Restricciones de caudal
for t in range(len(demanda_autoconsumo)):
    prob += (generacion[0] + generacion[1]) * 0.0009 + Qx + 0.2624 >= demanda_autoconsumo[t]
    prob += (generacion[0] + generacion[1]) * 0.0008 + Qx + 0.053 >= demanda_autoconsumo[t]
    prob += generacion[2] * 0.0005 - 0.0154 >= demanda_gran_consumidor[t]

# Resolver el problema
prob.solve()

# Imprimir resultado
print("Estado de la solución:", LpStatus[prob.status])
print("Generación óptima:")
for i, v in enumerate(generacion):
    print(f"Unidad {i+1}: {value(v)} kW")

print("Costo total de compra de energía:", value(costo_total))




AssertionError: ignored